## <div align="center">Create profile from start and end coordinates and sample raster along profile</div>

The Python™ tools in this notebook were developed for sampling SfM, ASP stereo and lidar bathymetery grids along a desired profile for comparison. Their basic functionality and verification are demsonstrated in this Jupyter notebook using a test surface elevation grid.

**Note:** Both, QGIS and `rasterio.sample` used here do NOT interpolate between neighboring grid cells, but return the value of the grid cell at the requested sample location. 

In [1]:
# load required modules, set output file name and processing switches

import pyproj
import rasterio
import numpy as np
import pandas as pd

# the two lines before importing GeoPandas are to turn off a warning message and tell GeoPandas to use Shapely instead of PyGEOS
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from   pyproj import Transformer

# file names
f_name_gpkg_raw = r"..\data\example_files\ice_divide_test_profile.gpkg"
f_name_gpkg_res = r"..\data\example_files\ice_divide_test_profile_srf.gpkg" 
f_name_geotiff  = r"..\data\example_files\Greenland_example_ice_surface_elevation_for_testing_1000m.tif"

VERBOSE = 2 # 0 = silent, 1 = displays profile information, 2 = displays profile information and WKT string

### Step 1: pick start and endpoints of desired profile  
For the purpose of this tutorial we will use a data set with EPSG:3413 as Coordinate Reference System (CRS) and define the start end end points of the profile interactively in a QGIS map project. Use the profile tool (ruler symbol) in QGIS for interactively picking start and end points.

In [2]:
# projected coordinates are in EPSG:3413 and picked in QGIS
x_e = -32217.228; y_e = -2499388.744;	
x_s = 158293.158; y_s = -2689040.009;

### Step 2: transform profile start and end points to lat lon

In [3]:
# EPSG:3413 NSIDC Sea Ice Polar Stereographic North/WGS-84 used for Greenland
# EPSG:4326 WGS84 - World Geodetic System 1984, used in GPS and DGPS trajectories

geo2xy = pyproj.Transformer.from_crs(4326,3413)
xy2geo = pyproj.Transformer.from_crs(3413,4326)

[lat_s,lon_s] = xy2geo.transform(x_s,y_s) 
[lat_e,lon_e] = xy2geo.transform(x_e,y_e) 

### Step 3: set up local map projection along to profile

In [4]:
# set up local map projection along to profile
local_map_proj = "+proj=tpeqd +lon_1 ={} +lat_1={} +lon_2={} +lat_2={} +x_0=0 y_0=0 +ellps=WGS84 +units=m +datum=WGS84 +no_defs".format(lon_s, lat_s, lon_e, lat_e)
crs_local = pyproj.CRS(local_map_proj)

# define local forward and inverse map projections
ll_to_xy = Transformer.from_crs("+proj=longlat +datum=WGS84 +no_defs",local_map_proj)
xy_to_ll = Transformer.from_crs(local_map_proj,"+proj=longlat +datum=WGS84 +no_defs")

### Step 4: transform profile start and end points to local projected coordinates

In [5]:
x_s, y_s = ll_to_xy.transform(lon_s, lat_s)
x_e, y_e = ll_to_xy.transform(lon_e, lat_e)

if VERBOSE == 1:
    print(f"Start : x_s: {x_s:+10.2f} m  |  x_e: {x_e:+10.2f} m")
    print(f"End   : y_s: {y_s:+10.2f} m  |  y_e: {y_e:+10.2f} m")    
    print(f"Length:                            {x_e - x_s:10.2f} m")

### Step 5: create equally spaced vertices along x-axis of profile and set up x,y coordinate arrays for export

In [6]:
# first determine number of points to roughly get desired spacing. Note: np.arange(x_s, y_s, 50) does not include end point. use np.linspace

spacing = 500.0 # approximate distance in meters between vertices
n_points = (x_e - x_s)/spacing  
n_points = int(np.ceil(n_points)) # needs to be rounded and converted to integer
    
x_array, x_spacing = np.linspace(x_s, x_e, num = n_points, endpoint=True, retstep=True) # includes end point and reports spacing

print(f"spacing : {x_spacing:.2f} m")
print(f"n_points: {len(x_array):d}")

# create array with y coordinates = 0.0 with same length as x_array, since y coordinates along great circle are only 0 for start and end points
y_array = x_array * 0.0

# create array with x_distance starting at profile start and same length as x_array
x_dist = x_array + x_e

spacing : 500.85 m
n_points: 530


### Step 6: reproject local projection to EPSG:3413

In [7]:
# first back to lat lon
lon, lat = xy_to_ll.transform(x_array, y_array)
# then lat lon to EPSG:3413
x, y = geo2xy.transform(lat, lon) # need to flip lat lon order

### Step 7: set up GeoDataFrame with point geometry

In [8]:
# create dataframe from scratch
profile_df = pd.DataFrame({'Longitude': x, 'Latitude': y, 'Distance': x_dist})

# replace lat lon geometry field with converted longitude in GeoDataFrame:
profile_gdf = gpd.GeoDataFrame(profile_df, geometry=gpd.points_from_xy(profile_df['Longitude'], profile_df['Latitude']))

# remove latitude and longitude columns since they are stored in geometry field
# mostly matters for large files -> faster load in QGIS and smaller file sizes
profile_gdf = profile_gdf.drop(columns=['Longitude'])
profile_gdf = profile_gdf.drop(columns=['Latitude'])

# set up the coordinate system for GeoDataPackage
profile_gdf = profile_gdf.set_crs("EPSG:3413")

### Step 8: save GeoDataPackage file

In [9]:
profile_gdf.to_file(f_name_gpkg_raw, driver="GPKG")

### Step 9: export WKT string with local map projection

In [10]:
if VERBOSE > 1:
    from osgeo import osr
    osr.UseExceptions()
    srs = osr.SpatialReference()
    srs.ImportFromProj4(local_map_proj)
    print("Well-known text (WKT) representation of local map projection along profile:\n")
    print(srs.ExportToWkt())

Well-known text (WKT) representation of local map projection along profile:

PROJCS["unknown",GEOGCS["unknown",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Two_Point_Equidistant"],PARAMETER["Latitude_Of_1st_Point",65.4989714571374],PARAMETER["Longitude_Of_1st_Point",-41.6311116126087],PARAMETER["Latitude_Of_2nd_Point",67.2183588300425],PARAMETER["Longitude_Of_2nd_Point",-45.7385041530438],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


### Step 10: load profile GeoDataPackage file in QGIS map project and sample raster file for verification data  
**Note:** you can skip this step and use the GeoDataPackage with resampled ice surface elevations provided in this repository.  

QGIS instructions:
1) Project -> New: select "./data/example_files/Greenland_example_ice_surface_elevation_for_testing_1000m.tif"
2) Layer -> Add Layer -> Add Raster Layer: select "./data/example_files/ice_divide_test_profile.gpkg"
3) Processing Toolbox -> Raster analysis -> Sample raster values: save as "./data/example_files/ice_divide_test_profile_srf.gpkg" 

### Step 11: load profile GeoDataPackage file with QGIS resampled surface elevation values as test data

In [11]:
profile_gdf = gpd.read_file(f_name_gpkg_res)
coord_list = [(x,y) for x,y in zip(profile_gdf['geometry'].x , profile_gdf['geometry'].y)]

### Step 12: compare QGIS and Python surface elevations resampled from GeoTiff raster file

In [12]:
resampled_gdf = profile_gdf.copy()  # create copy of original GDF and keep original unchanged
with rasterio.open(f_name_geotiff) as src:
    resampled_gdf['srf_int'] = [x[0] for x in src.sample(coord_list)]

diff = np.asarray(resampled_gdf.srf_int - resampled_gdf.SRF_1)

# display comparison
print(f"Δz min : {np.min(diff):+4.6f} m")
print(f"Δz max : {np.max(diff):+4.6f} m")
print(f"Δz mean: {np.mean(diff):+4.6f} m")
print(f"n_pts  : {len(diff):d}")

Δz min : +0.000000 m
Δz max : +0.000000 m
Δz mean: +0.000000 m
n_pts  : 530


In [13]:
print(resampled_gdf)

         Distance        SRF_1                         geometry      srf_int
0   -1.699664e-08  1656.426880  POINT (158293.062 -2689040.106)  1656.426880
1    5.008539e+02  1656.426880  POINT (157930.822 -2688681.488)  1656.426880
2    1.001708e+03  1678.612915  POINT (157568.398 -2688323.064)  1678.612915
3    1.502562e+03  1678.612915  POINT (157206.176 -2687964.446)  1678.612915
4    2.003416e+03  1695.448120  POINT (156843.769 -2687606.022)  1695.448120
..            ...          ...                              ...          ...
525  2.629483e+05  2121.612793  POINT (-30785.234 -2500822.902)  2121.612793
526  2.634492e+05  2121.637207  POINT (-31143.341 -2500464.459)  2121.637207
527  2.639500e+05  2117.739014  POINT (-31501.247 -2500105.823)  2117.739014
528  2.644509e+05  2115.763184  POINT (-31859.241 -2499747.284)  2115.763184
529  2.649517e+05  2108.476074  POINT (-32217.228 -2499388.744)  2108.476074

[530 rows x 4 columns]
